In [1]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Config, MT5EncoderModel, MT5Tokenizer, Trainer, TrainingArguments
from progeny_tokenizer import TAPETokenizer
import numpy as np
import math
import random
import scipy
import time
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, Dataset, BatchSampler
import typing
from pathlib import Path
import argparse
from collections import OrderedDict
import pickle
import pathlib

import matplotlib.pyplot as plt

In [2]:
noperturb_results_tsv = 'generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_noperturb_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_noperturb-congen_seqs25000.tsv'

In [3]:
perturb_results_tsvs = [
    'generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0035_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0035-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb007_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb007-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb017_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb017-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb035_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb035-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb087_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb087-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb175_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb175-congen_seqs25000.tsv'

]

In [4]:
noperturb_df = pd.read_table(noperturb_results_tsv)

In [5]:
noperturb_df = noperturb_df.sort_values(by='latent_head_pred', ascending=False)
noperturb_df

,disc_pred,latent_head_pred,gen_input_seq_class,gt_class_pred,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq_ppl,input_seq_ppl,generated_seq,gen_input_seq,repeated_gen,in_train_data_gen
0,5.201600,7.116148,2,4,0.992559,1,0.000154,0.996442,69.147690,41.990410,It's definitely a strikingly complex yet subtl...,"It 's not particularly subtle ... However , it...",False,False
1,4.907716,7.077060,2,3,0.999937,1,0.000040,0.999947,721.473083,231.790436,An often untold but often enjoyable outing.,An often unfunny romp .,False,False
2,4.693709,7.060658,2,3,0.999947,1,0.000028,0.999957,143.099060,80.846931,"The soupy distinction is fun, but it can also ...",The soupy end result has the odd distinction o...,False,False
3,4.866645,6.949634,2,3,0.999931,1,0.000035,0.999939,180.744675,231.790436,An often fanciful but sometimes surprisingly p...,An often unfunny romp .,False,False
4,5.022109,6.920750,2,3,0.999959,1,0.000021,0.999966,1118.118896,96.533356,Clever but distinctly compelling.,Clever but not especially compelling .,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25046,2.424020,-4.042237,2,1,0.999768,0,0.999977,0.000017,232.807404,154.913544,"A rip-off of a cinematic experience, only full...",A pure participatory event that malnourished i...,False,False
25047,3.385083,-4.279074,2,1,0.999956,0,0.999966,0.000016,124.039375,272.090424,Feels totally dreary and uncanny.,Feels untidily honest .,False,False
25048,2.701180,-5.026286,2,0,0.999321,0,0.999948,0.000042,76.747101,272.090424,Feels unfunny and unflattering.,Feels untidily honest .,False,False
25049,3.294072,-5.180806,2,1,0.997851,0,0.999929,0.000064,1106.568359,399.065369,Adam Sandler is THE BAD Adam SHARKLER.,Adam SANDLER !,False,False


In [6]:
noperturb_df['sent_delta'] = noperturb_df['gt_class_pred'] - noperturb_df['gen_input_seq_class']
noperturb_df

,disc_pred,latent_head_pred,gen_input_seq_class,gt_class_pred,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq_ppl,input_seq_ppl,generated_seq,gen_input_seq,repeated_gen,in_train_data_gen,sent_delta
0,5.201600,7.116148,2,4,0.992559,1,0.000154,0.996442,69.147690,41.990410,It's definitely a strikingly complex yet subtl...,"It 's not particularly subtle ... However , it...",False,False,2
1,4.907716,7.077060,2,3,0.999937,1,0.000040,0.999947,721.473083,231.790436,An often untold but often enjoyable outing.,An often unfunny romp .,False,False,1
2,4.693709,7.060658,2,3,0.999947,1,0.000028,0.999957,143.099060,80.846931,"The soupy distinction is fun, but it can also ...",The soupy end result has the odd distinction o...,False,False,1
3,4.866645,6.949634,2,3,0.999931,1,0.000035,0.999939,180.744675,231.790436,An often fanciful but sometimes surprisingly p...,An often unfunny romp .,False,False,1
4,5.022109,6.920750,2,3,0.999959,1,0.000021,0.999966,1118.118896,96.533356,Clever but distinctly compelling.,Clever but not especially compelling .,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25046,2.424020,-4.042237,2,1,0.999768,0,0.999977,0.000017,232.807404,154.913544,"A rip-off of a cinematic experience, only full...",A pure participatory event that malnourished i...,False,False,-1
25047,3.385083,-4.279074,2,1,0.999956,0,0.999966,0.000016,124.039375,272.090424,Feels totally dreary and uncanny.,Feels untidily honest .,False,False,-1
25048,2.701180,-5.026286,2,0,0.999321,0,0.999948,0.000042,76.747101,272.090424,Feels unfunny and unflattering.,Feels untidily honest .,False,False,-2
25049,3.294072,-5.180806,2,1,0.997851,0,0.999929,0.000064,1106.568359,399.065369,Adam Sandler is THE BAD Adam SHARKLER.,Adam SANDLER !,False,False,-1


In [7]:
noperturb_df['ppl_delta'] = noperturb_df['generated_seq_ppl'] - noperturb_df['input_seq_ppl']
noperturb_df

,disc_pred,latent_head_pred,gen_input_seq_class,gt_class_pred,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq_ppl,input_seq_ppl,generated_seq,gen_input_seq,repeated_gen,in_train_data_gen,sent_delta,ppl_delta
0,5.201600,7.116148,2,4,0.992559,1,0.000154,0.996442,69.147690,41.990410,It's definitely a strikingly complex yet subtl...,"It 's not particularly subtle ... However , it...",False,False,2,27.157280
1,4.907716,7.077060,2,3,0.999937,1,0.000040,0.999947,721.473083,231.790436,An often untold but often enjoyable outing.,An often unfunny romp .,False,False,1,489.682648
2,4.693709,7.060658,2,3,0.999947,1,0.000028,0.999957,143.099060,80.846931,"The soupy distinction is fun, but it can also ...",The soupy end result has the odd distinction o...,False,False,1,62.252129
3,4.866645,6.949634,2,3,0.999931,1,0.000035,0.999939,180.744675,231.790436,An often fanciful but sometimes surprisingly p...,An often unfunny romp .,False,False,1,-51.045761
4,5.022109,6.920750,2,3,0.999959,1,0.000021,0.999966,1118.118896,96.533356,Clever but distinctly compelling.,Clever but not especially compelling .,False,False,1,1021.585541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25046,2.424020,-4.042237,2,1,0.999768,0,0.999977,0.000017,232.807404,154.913544,"A rip-off of a cinematic experience, only full...",A pure participatory event that malnourished i...,False,False,-1,77.893860
25047,3.385083,-4.279074,2,1,0.999956,0,0.999966,0.000016,124.039375,272.090424,Feels totally dreary and uncanny.,Feels untidily honest .,False,False,-1,-148.051048
25048,2.701180,-5.026286,2,0,0.999321,0,0.999948,0.000042,76.747101,272.090424,Feels unfunny and unflattering.,Feels untidily honest .,False,False,-2,-195.343323
25049,3.294072,-5.180806,2,1,0.997851,0,0.999929,0.000064,1106.568359,399.065369,Adam Sandler is THE BAD Adam SHARKLER.,Adam SANDLER !,False,False,-1,707.502991


# Compute % target class for noperturb

In [8]:
target_classes = [3,4]
topk_list = [25000, 10000, 1000, 100, 10]

percent_target_class = []

In [9]:
gt_class_preds = noperturb_df['gt_class_pred']
gen_input_seq_classes = noperturb_df['gen_input_seq_class']
sent_deltas = noperturb_df['sent_delta']

generated_seq_ppls = noperturb_df['generated_seq_ppl']
input_seq_ppls = noperturb_df['input_seq_ppl']
ppl_deltas = noperturb_df['ppl_delta']

In [10]:
df = noperturb_df

In [11]:
for target_class in target_classes:
    total_num  = len(df['gt_class_pred'])
    print("target_class: ", target_class)
    num_target_class = np.sum(df['gt_class_pred'] == target_class)
    
    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)
    
    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
        topk_sent_deltas = sent_deltas[:topk]
        
        
        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
        topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
        topk_percent_target_class = topk_num_target_class / topk_num *100
        print("topk_percent_target_class: ", topk_percent_target_class)
    
        topk_sent_delta_mean = np.mean(topk_sent_deltas)
        print("topk_sent_delta_mean: ", topk_sent_delta_mean)
        
        print("*")
    print("--------------")
    
    
print("-------For all target classes-------")
print("target_classes: ", target_classes)
total_num  = len(df['gt_class_pred'])
num_target_class = np.sum(df['gt_class_pred'].isin(target_classes))

percent_target_class = num_target_class / total_num *100
print("percent_target_class: ", percent_target_class)

for topk in topk_list:
    topk_gt_class_preds = gt_class_preds[:topk]
    topk_sent_deltas = sent_deltas[:topk]
    
    topk_generated_seq_ppls = generated_seq_ppls[:topk]
    topk_ppl_deltas = ppl_deltas[:topk]

    topk_num  = len(topk_gt_class_preds)
    print("topk: ", topk)
    topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
    topk_percent_target_class = topk_num_target_class / topk_num *100
    print("topk_percent_target_class: ", topk_percent_target_class)

    topk_sent_delta_mean = np.mean(topk_sent_deltas)
    print("topk_sent_delta_mean: ", topk_sent_delta_mean)
    
    
    topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
    topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
    print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
    print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)
    
    topk_ppl_delta_mean = np.mean(topk_ppl_deltas)
    topk_ppl_delta_std = np.std(topk_ppl_deltas)
    print("topk_ppl_delta_mean: ", topk_ppl_delta_mean)
    print("topk_ppl_delta_std: ", topk_ppl_delta_std)

    print("*")

target_class:  3
percent_target_class:  39.291844636940645
topk:  25000
topk_percent_target_class:  39.372
topk_sent_delta_mean:  0.44156
*
topk:  10000
topk_percent_target_class:  56.21000000000001
topk_sent_delta_mean:  0.8295
*
topk:  1000
topk_percent_target_class:  61.8
topk_sent_delta_mean:  1.022
*
topk:  100
topk_percent_target_class:  67.0
topk_sent_delta_mean:  0.95
*
topk:  10
topk_percent_target_class:  70.0
topk_sent_delta_mean:  1.1
*
--------------
target_class:  4
percent_target_class:  7.424853299269491
topk:  25000
topk_percent_target_class:  7.4399999999999995
topk_sent_delta_mean:  0.44156
*
topk:  10000
topk_percent_target_class:  13.84
topk_sent_delta_mean:  0.8295
*
topk:  1000
topk_percent_target_class:  20.3
topk_sent_delta_mean:  1.022
*
topk:  100
topk_percent_target_class:  14.000000000000002
topk_sent_delta_mean:  0.95
*
topk:  10
topk_percent_target_class:  20.0
topk_sent_delta_mean:  1.1
*
--------------
-------For all target classes-------
target_classes

# iterate through all perturbed result tsv files

In [12]:
for perturb_results_tsv in perturb_results_tsvs:
    print("*-"*30)
    print("perturb_results_tsv: ", perturb_results_tsv)
    
    perturb_df = pd.read_table(perturb_results_tsv)
    perturb_df = perturb_df.sort_values(by='latent_head_pred', ascending=False)
    perturb_df['sent_delta'] = perturb_df['gt_class_pred'] - perturb_df['gen_input_seq_class']
    perturb_df['ppl_delta'] = perturb_df['generated_seq_ppl'] - perturb_df['input_seq_ppl']

    gt_class_preds = perturb_df['gt_class_pred']
    gen_input_seq_classes = perturb_df['gen_input_seq_class']
    sent_deltas = perturb_df['sent_delta']
    
    generated_seq_ppls = perturb_df['generated_seq_ppl']
    input_seq_ppls = perturb_df['input_seq_ppl']
    ppl_deltas = perturb_df['ppl_delta']

    for target_class in target_classes:
        total_num  = len(perturb_df['gt_class_pred'])
        print("target_class: ", target_class)
        num_target_class = np.sum(perturb_df['gt_class_pred'] == target_class)

        percent_target_class = num_target_class / total_num *100
        print("percent_target_class: ", percent_target_class)

        for topk in topk_list:
            topk_gt_class_preds = gt_class_preds[:topk]
            topk_sent_deltas = sent_deltas[:topk]


            topk_num  = len(topk_gt_class_preds)
            print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
            topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
            topk_percent_target_class = topk_num_target_class / topk_num *100
#             print("topk_num_target_class: ", topk_num_target_class)
#             print("topk_num: ", topk_num)
            print("topk_percent_target_class: ", topk_percent_target_class)

            topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
            print("topk_sent_delta_mean: ", topk_sent_delta_mean)

            print("*")
        print("--------------")


    print("-------For all target classes-------")
    print("target_classes: ", target_classes)
    total_num  = len(perturb_df['gt_class_pred'])
    num_target_class = np.sum(perturb_df['gt_class_pred'].isin(target_classes))

    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)

    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
        topk_sent_deltas = sent_deltas[:topk]
    
        topk_generated_seq_ppls = generated_seq_ppls[:topk]
        topk_ppl_deltas = ppl_deltas[:topk]

        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
        topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
        topk_percent_target_class = topk_num_target_class / topk_num *100
#         print("topk_num_target_class: ", topk_num_target_class)
#         print("topk_num: ", topk_num)
        print("topk_percent_target_class: ", topk_percent_target_class)

        topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
        print("topk_sent_delta_mean: ", topk_sent_delta_mean)

    
        topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
        topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
        print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
        print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)

        topk_ppl_delta_mean = np.mean(topk_ppl_deltas)
        topk_ppl_delta_std = np.std(topk_ppl_deltas)
        print("topk_ppl_delta_mean: ", topk_ppl_delta_mean)
        print("topk_ppl_delta_std: ", topk_ppl_delta_std)

        print("*")

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
perturb_results_tsv:  generated_seqs/congen_SST5/SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0035_trainlabel2geninput/16xgen25kunique_SST5_leave34out_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0035-congen_seqs25000.tsv
target_class:  3
percent_target_class:  40.3636073678335
topk:  25000
topk_percent_target_class:  40.492
topk_sent_delta_mean:  0.46648
*
topk:  10000
topk_percent_target_class:  56.599999999999994
topk_sent_delta_mean:  0.8494
*
topk:  1000
topk_percent_target_class:  63.5
topk_sent_delta_mean:  0.993
*
topk:  100
topk_percent_target_class:  76.0
topk_sent_delta_mean:  1.0
*
topk:  10
topk_percent_target_class:  80.0
topk_sent_delta_mean:  0.8
*
--------------
target_class:  4
percent_target_class:  7.754565026712383
topk:  25000
topk_percent_target_class:  7.779999999999999
topk_sent_delta_mean:  0.46648
*
topk:  1000

target_class:  3
percent_target_class:  46.191235059760956
topk:  25000
topk_percent_target_class:  46.36
topk_sent_delta_mean:  0.5348
*
topk:  10000
topk_percent_target_class:  56.36
topk_sent_delta_mean:  0.771
*
topk:  1000
topk_percent_target_class:  69.0
topk_sent_delta_mean:  1.07
*
topk:  100
topk_percent_target_class:  68.0
topk_sent_delta_mean:  1.06
*
topk:  10
topk_percent_target_class:  90.0
topk_sent_delta_mean:  1.1
*
--------------
target_class:  4
percent_target_class:  5.940239043824701
topk:  25000
topk_percent_target_class:  5.9639999999999995
topk_sent_delta_mean:  0.5348
*
topk:  10000
topk_percent_target_class:  10.870000000000001
topk_sent_delta_mean:  0.771
*
topk:  1000
topk_percent_target_class:  19.0
topk_sent_delta_mean:  1.07
*
topk:  100
topk_percent_target_class:  19.0
topk_sent_delta_mean:  1.06
*
topk:  10
topk_percent_target_class:  10.0
topk_sent_delta_mean:  1.1
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_ta